# Agente de Informes de Gastos con Microsoft Agent Framework

## Descripción

En este ejercicio, crearás un **agente de informes de gastos empresariales** utilizando Microsoft Agent Framework. El agente procesará datos de gastos corporativos generales y enviará informes automáticos al departamento de finanzas.

### Cargar librerías

In [ ]:
import os
import asyncio
from pathlib import Path
from agent_framework import AgentThread, ChatAgent
from agent_framework.openai import OpenAIChatClient
from pydantic import Field
from typing import Annotated

# GitHub Models client configuration
MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

### Visualizar los datos de gastos

In [ ]:
script_dir = Path.cwd()
file_path = script_dir / 'data.txt'
with file_path.open('r') as file:
    expenses_data = file.read()
    print("Datos de gastos:")
    print(expenses_data)

### Definir una función de herramienta (Tool Function)

Creamos una función `send_email` que el agente podrá utilizar. Esta función simula el envío de un correo electrónico imprimiendo los detalles en la consola.

In [ ]:
def send_email(
    to: Annotated[str, Field(description="Email recipient")],
    subject: Annotated[str, Field(description="Email subject")],
    body: Annotated[str, Field(description="Email body")]):
        print("\n--- Simulated Email ---")
        print("To:", to)
        print("Subject:", subject)
        print("Body:")
        print(body)
        print("----------------------")

print("Function 'send_email' defined")

### Crear y ejecutar el agente

Ahora, creamos un `ChatAgent` con instrucciones específicas y le proporcionamos la función `send_email` como herramienta. Luego, le enviamos los datos de gastos junto con un prompt.

In [ ]:
async def process_expenses(prompt, data):
    print("Creating expenses agent...")
    
    # GitHub Models client configuration
    client = OpenAIChatClient(
        model_id=MODEL_NAME,
        api_key=os.environ["GITHUB_TOKEN"],
        base_url="https://models.github.ai/inference"
    )
    
    async with ChatAgent(
        chat_client=client,
        name="expenses_agent",
        instructions="""
		Eres un asistente de IA para informes de gastos corporativos de la empresa.
		Cuando un usuario envíe datos de gastos empresariales (viajes de negocios, formación, compras de oficina,
		eventos corporativos o material de trabajo) y solicite una reclamación, usa la función disponible para enviar un correo a finance@example.com
		con el asunto 'Informe de Gastos - Operaciones Corporativas' y un cuerpo que contenga:
		- Gastos desglosados por categoría y destino
		- Propósito del gasto o referencia de proyecto
		- Importe total en EUR
		Luego confirma al usuario que el informe de gastos ha sido enviado.
        """,
        tools=send_email,
    ) as agent:
        try:
            print(f"Processing expense data for: {prompt}")
            prompt_messages = [f"{prompt}: {data}"]
            print("Running agent...")
            response = await agent.run(prompt_messages)
            print("\nAgent response:")
            print(f"{response}")
        except Exception as e:
            print(f"Error occurred: {e}")

# Execute processing
print("Starting expense processing...")
user_prompt = "Enviar informe de gastos del último viaje de negocios"
await process_expenses(user_prompt, expenses_data)
print("Expense processing completed")